<a href="https://colab.research.google.com/github/glebmikha/ds-course/blob/main/08_useful_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Мой Telegram-канал: https://t.me/mikhaylovgleb

# Загрузка данных

https://www.kaggle.com/shubh0799/churn-modelling

https://drive.google.com/file/d/1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ/view?usp=sharing

In [1]:
! gdown --id 1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ

Downloading...
From: https://drive.google.com/uc?id=1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ
To: /content/churn-modeling.zip
100% 268k/268k [00:00<00:00, 71.8MB/s]


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/churn-modeling.zip')

# Обучающая, валидационная и тестовая выборка

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(df,train_size=0.6,random_state=42,stratify=df['Exited'])

In [6]:
val, test = train_test_split(test,train_size=0.5,random_state=42,stratify=test['Exited'])

# Обучение модели

In [7]:
! pip install catboost

     |████████████████████████████████| 76.1 MB 1.4 MB/s 


In [8]:
from catboost import CatBoostClassifier

In [9]:
X = ['CustomerId', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

cat_features = ['Geography','Gender']

y = ['Exited']

In [10]:
from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                 )

test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )

In [11]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [12]:
model = CatBoostClassifier(**params)

In [13]:
model.fit(train_data,eval_set=valid_data)

0:	test: 0.8338545	best: 0.8338545 (0)	total: 72.8ms	remaining: 1m 12s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 1.56s	remaining: 13.9s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 2.34s	remaining: 9.31s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 3.13s	remaining: 7.27s
400:	test: 0.8823868	best: 0.8824376 (390)	total: 4.1s	remaining: 6.13s
500:	test: 0.8834491	best: 0.8834522 (498)	total: 4.9s	remaining: 4.88s
600:	test: 0.8834737	best: 0.8836554 (564)	total: 5.71s	remaining: 3.79s
700:	test: 0.8837847	best: 0.8838694 (685)	total: 6.48s	remaining: 2.76s
800:	test: 0.8834383	best: 0.8838694 (685)	total: 7.25s	remaining: 1.8s
900:	test: 0.8829318	best: 0.8838694 (685)	total: 8.09s	remaining: 888ms
999:	test: 0.8822744	best: 0.8838694 (685)	total: 9.95s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685

Shrink model to first 686 iterations.


In [14]:
n_iters = model.best_iteration_ + 1

In [15]:
n_iters

686

In [16]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [17]:
model = CatBoostClassifier(**params)

In [18]:
train_full = pd.concat([train,val])

In [19]:
train_full_data = Pool(train_full[X],
                       label=train_full[y],
                       cat_features=cat_features)

In [20]:
model.fit(train_full_data)

0:	total: 47.3ms	remaining: 32.4s
100:	total: 1.71s	remaining: 9.9s
200:	total: 2.68s	remaining: 6.47s
300:	total: 3.67s	remaining: 4.7s
400:	total: 5.19s	remaining: 3.69s
500:	total: 7.2s	remaining: 2.66s
600:	total: 9.41s	remaining: 1.33s
685:	total: 11.4s	remaining: 0us


In [21]:
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1]

In [22]:
from sklearn.metrics import roc_auc_score

In [23]:
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])

0.8735839074822127

# Кросс-валидация

https://github.com/catboost/tutorials/blob/master/classification/classification_tutorial.ipynb

In [24]:
from catboost import cv

In [25]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [26]:
cv_data = cv(
    params = params,
    pool = train_full_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False
)

Training on fold [0/5]

bestTest = 0.866360207
bestIteration = 881

Training on fold [1/5]

bestTest = 0.8716621864
bestIteration = 998

Training on fold [2/5]

bestTest = 0.8695412245
bestIteration = 540

Training on fold [3/5]

bestTest = 0.8795945701
bestIteration = 942

Training on fold [4/5]

bestTest = 0.8586794872
bestIteration = 750



In [27]:
cv_data

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.818479,0.009480,0.685020,0.000209,0.684939,0.000262
1,1,0.823848,0.010334,0.677216,0.000374,0.677055,0.000503
2,2,0.831819,0.009265,0.668989,0.000472,0.668799,0.000751
3,3,0.834559,0.008771,0.662008,0.001326,0.661753,0.000689
4,4,0.834483,0.008218,0.654599,0.001429,0.654270,0.000898
...,...,...,...,...,...,...,...
995,995,0.868703,0.007809,0.329986,0.016873,0.284097,0.003644
996,996,0.868715,0.007781,0.329972,0.016875,0.284057,0.003631
997,997,0.868718,0.007804,0.329972,0.016876,0.284003,0.003636
998,998,0.868725,0.007777,0.329969,0.016855,0.283943,0.003613


In [28]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
741,741,0.868822,0.007457,0.330818,0.017005,0.297428,0.004326


In [29]:
n_iters = cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]['iterations'].values[0]

In [30]:
n_iters

741

In [31]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [32]:
model = CatBoostClassifier(**params)

In [33]:
model.fit(train_full_data)

0:	total: 19.8ms	remaining: 14.6s
100:	total: 964ms	remaining: 6.11s
200:	total: 1.88s	remaining: 5.04s
300:	total: 2.77s	remaining: 4.04s
400:	total: 3.67s	remaining: 3.11s
500:	total: 4.56s	remaining: 2.18s
600:	total: 5.43s	remaining: 1.26s
700:	total: 6.34s	remaining: 362ms
740:	total: 6.69s	remaining: 0us


In [34]:
test['y_score_cross_val'] = model.predict_proba(test_data)[:,1]

In [35]:
from sklearn.metrics import roc_auc_score

In [36]:
roc_auc_score(test['Exited'],test['y_score_cross_val'])

0.8737196364315009

In [37]:
len(train_full)

8000

# Подбор гиперпараметров

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/catboost_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=2043

https://github.com/catboost/catboost/blob/master/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning.ipynb

In [38]:
model.get_all_params()

{'auto_class_weights': 'None',
 'bayesian_matrix_reg': 0.10000000149011612,
 'best_model_min_trees': 1,
 'boost_from_average': False,
 'boosting_type': 'Plain',
 'bootstrap_type': 'MVS',
 'border_count': 254,
 'class_names': [0, 1],
 'classes_count': 0,
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'counter_calc_method': 'SkipTest',
 'ctr_leaf_count_limit': 18446744073709551615,
 'ctr_target_border_count': 1,
 'depth': 6,
 'eval_metric': 'AUC',
 'feature_border_type': 'GreedyLogSum',
 'fold_permutation_block': 0,
 'force_unit_auto_pair_weights': False,
 'grow_policy': 'SymmetricTree',
 'has_time': False,
 'iterations': 741,
 'l2_leaf_reg': 3,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'leaf_estimation_iterations': 10,
 'leaf_estimation_method': 'Newton',
 'learning_rate': 0.009999999776482582,
 'loss_functio

In [39]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42}

In [40]:
model = CatBoostClassifier(**params)

In [41]:
grid = {'learning_rate': [0.01, 0.1],
        'depth': [5, 6]}

In [42]:
result = model.grid_search(grid, train_full_data, verbose=False)

0:	test: 0.7811641	best: 0.7811641 (0)	total: 9.35ms	remaining: 9.34s
100:	test: 0.8435462	best: 0.8435769 (99)	total: 816ms	remaining: 7.26s
200:	test: 0.8508923	best: 0.8508923 (200)	total: 1.59s	remaining: 6.33s
300:	test: 0.8547897	best: 0.8547897 (300)	total: 2.36s	remaining: 5.47s
400:	test: 0.8569385	best: 0.8569385 (400)	total: 3.12s	remaining: 4.66s
500:	test: 0.8576410	best: 0.8576410 (500)	total: 3.91s	remaining: 3.89s
600:	test: 0.8585538	best: 0.8585538 (600)	total: 4.66s	remaining: 3.1s
700:	test: 0.8590923	best: 0.8591487 (694)	total: 5.41s	remaining: 2.31s
800:	test: 0.8593590	best: 0.8595564 (749)	total: 6.16s	remaining: 1.53s
900:	test: 0.8593282	best: 0.8595564 (749)	total: 6.93s	remaining: 761ms
999:	test: 0.8593897	best: 0.8595974 (986)	total: 7.69s	remaining: 0us

bestTest = 0.8595974359
bestIteration = 986

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	test: 0.7811641	best:

In [47]:
result['params']

{'depth': 5, 'learning_rate': 0.1}

In [48]:
pd.DataFrame(result['cv_results'])['test-AUC-mean'].max()

0.8689660773549619

# Принципы sklearn

https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

# Преодбработка и фичеинжениринг

https://catboost.ai/en/docs/concepts/quantization

https://www.kaggle.com/learn/feature-engineering

# Калибровка

https://www.kaggle.com/residentmario/notes-on-classification-probability-calibration

# Catboost, тексты и эмбеддинги

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/text_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=3802

# Главные навыки для аналитика

- SQL
- pandas
- Основы статистики (AB тесты)

# Главные навыки для Дата Саентиста

- Все то же, что и у аналитика
- Машинное обучение (sklearn, catboost)
- более продвинутый python (основы ООП)
- linux, git, docker, веб-сервисы

# Как развиваться дальше?

- [Прикладное машинное обучение с помощью Scikit-Learn, Keras и TensorFlow: концепции, инструменты и техники для создания интеллектуальных систем, 2-е издание | Жерон Орельен](https://www.ozon.ru/product/prikladnoe-mashinnoe-obuchenie-s-pomoshchyu-scikit-learn-keras-i-tensorflow-kontseptsii-207392052/?sh=HC92ZQyhUQ)
- Задачи на kaggle, вот [список](https://www.kaggle.com/getting-started/114864) для начинающих
- Откликайся на вакансии и решай тестовые. Можешь смотреть [тестовые по дс у меня на канале](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbefSg1XN3FA-SdSRFcCQfn)
- [Мой курс по статистике](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbHLLI66KdscKp_FJt0FsIi)
- Задачи по pandas и sql на [StrataScratch](https://www.stratascratch.com/?via=gleb)
- [Мой курс по SQL](https://www.udemy.com/course/sql-with-gleb/?referralCode=C99EC81AE75FEC9F50A2)
- Задачи на [itresume](https://itresume.ru/)
- Задачи на [Checkio](https://checkio.org/)
- Задачи на [Leetcode](https://leetcode.com/) вот по этому [списку](https://seanprashad.com/leetcode-patterns/)
- Мой [плейлист](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xZ4qiXlmQQLgAPyXJhQ7HxZ) по основам Linux
- [Основы Git](https://youtu.be/0cGIiA0AjNw)
- [Основы докера](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbR0ka2TUxJkXna40XWu92m)
- [Основы веб-сервисов](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xYLEAC5Y_sKqrJ9RA-U7Dja)

# Дополнительные материалы

- [Как работает градиентный бустинг](https://youtu.be/ZNJ3lKyI-EY)
- [Как учить английский](https://youtu.be/f9Q4Mwvd5pc)
- [Как искать работу](https://deepnote.com/@gleb-mikhaylov-15d6/blog-posts-nXDqO5CTQYOreNE53H3y-A)

Мой Telegram-канал: https://t.me/mikhaylovgleb